In [1]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import pytz
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from sklearn.linear_model import Lasso
from scipy.optimize import minimize

In [12]:
# Set random seed for reproducibility
np.random.seed(69)

engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")

# # Display the first few rows of the dataframe
# stock_returns.head()

# # Calculate the mean returns (expected returns)
# expected_returns = stock_returns.mean()

# # Calculate the covariance matrix of returns
# cov_matrix = stock_returns.cov()

# # Display the expected returns and covariance matrix
# expected_returns, cov_matrix


# from sklearn.linear_model import Lasso
# from scipy.optimize import minimize

# # Define risk-free rate (e.g., 1% annualized)
# risk_free_rate = 0.01 / 252  # Converting annualized to daily

# # Define the Sharpe ratio objective function to minimize
# def sharpe_ratio_neg(weights, expected_returns, cov_matrix, risk_free_rate):
#     # Portfolio return
#     port_return = np.dot(weights, expected_returns) - risk_free_rate
#     # Portfolio volatility
#     port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
#     # Sharpe ratio (negative for minimization)
#     return -port_return / port_volatility

# # Constraint: sum of weights must be 1 (fully invested portfolio)
# constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

# # Bounds for weights: between 0 and 1
# bounds = [(0, 1) for _ in range(len(tickers))]

# # Initial guess for weights
# initial_weights = np.ones(len(tickers)) / len(tickers)

# # Lasso regression with a certain alpha (regularization parameter)
# lasso = Lasso(alpha=0.01, max_iter=10000)

# # Fit the lasso model (it requires reshaping the inputs)
# lasso.fit(expected_returns.values.reshape(-1, 1), initial_weights)

# # Get the initial lasso weights
# lasso_weights = lasso.coef_

# # Perform the optimization
# result = minimize(sharpe_ratio_neg, lasso_weights, args=(expected_returns, cov_matrix, risk_free_rate),
#                   method='SLSQP', bounds=bounds, constraints=constraints)

# # Optimal portfolio weights
# optimal_weights = result.x

# # Display the optimal weights
# optimal_weights

In [13]:
stock_returns = pd.read_sql("SELECT * FROM stock_returns", con = engine)

In [14]:
stock_returns = stock_returns.set_index('index')
net_returns = stock_returns - 1

In [16]:
expected_returns = net_returns.mean()

In [17]:
expected_returns

NVDA    0.002665
TSLA    0.001601
AAPL    0.001029
AMD     0.002081
MSFT    0.001026
          ...   
MLI     0.000855
TLRY    0.000454
EXEL    0.001345
UGI     0.000012
HOLO    0.003733
Length: 1000, dtype: float64

In [18]:
cov_matrix = net_returns.cov()

In [19]:
cov_matrix

,NVDA,TSLA,AAPL,AMD,MSFT,META,SMCI,AMZN,GOOGL,AVGO,...,CVLT,SNV,AGI,BE,AMC,MLI,TLRY,EXEL,UGI,HOLO
NVDA,0.000920,0.000441,0.000295,0.000647,0.000312,-0.000073,0.000412,0.000317,0.000287,0.000416,...,0.000222,0.000227,0.000048,0.000401,0.000271,0.000222,0.000306,0.000234,0.000088,0.000118
TSLA,0.000441,0.001251,0.000272,0.000441,0.000245,0.000021,0.000275,0.000285,0.000231,0.000328,...,0.000202,0.000247,0.000086,0.000531,0.000406,0.000212,0.000394,0.000206,0.000107,0.000149
AAPL,0.000295,0.000272,0.000323,0.000276,0.000209,-0.000147,0.000171,0.000205,0.000193,0.000234,...,0.000131,0.000157,0.000036,0.000211,0.000190,0.000150,0.000162,0.000151,0.000086,0.000005
AMD,0.000647,0.000441,0.000276,0.001358,0.000263,-0.000066,0.000378,0.000305,0.000251,0.000372,...,0.000231,0.000225,0.000098,0.000344,0.000238,0.000211,0.000334,0.000259,0.000099,0.000093
MSFT,0.000312,0.000245,0.000209,0.000263,0.000292,-0.000048,0.000185,0.000228,0.000217,0.000220,...,0.000143,0.000151,0.000034,0.000204,0.000184,0.000141,0.000130,0.000148,0.000086,0.000063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MLI,0.000222,0.000212,0.000150,0.000211,0.000141,-0.000067,0.000218,0.000116,0.000138,0.000205,...,0.000160,0.000336,0.000090,0.000313,0.000227,0.000469,0.000236,0.000145,0.000177,0.000074
TLRY,0.000306,0.000394,0.000162,0.000334,0.000130,-0.000295,0.000286,0.000185,0.000183,0.000249,...,0.000206,0.000406,0.000152,0.000628,0.000692,0.000236,0.003071,0.000183,0.000177,-0.000081
EXEL,0.000234,0.000206,0.000151,0.000259,0.000148,-0.000080,0.000149,0.000148,0.000136,0.000172,...,0.000150,0.000185,-0.000016,0.000186,0.000202,0.000145,0.000183,0.001167,0.000075,-0.000311
UGI,0.000088,0.000107,0.000086,0.000099,0.000086,-0.000024,0.000113,0.000054,0.000083,0.000113,...,0.000093,0.000229,0.000031,0.000193,0.000186,0.000177,0.000177,0.000075,0.000302,-0.000080


In [24]:
risk_free_rates = pd.read_sql("SELECT * FROM risk_free_rate", con=engine)

In [25]:
risk_free_rates

,index
